In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import lfilter,correlate
import tensorflow.keras.layers as tfl
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau
from PIL import Image

2024-06-22 18:54:37.568408: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 18:54:37.568490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 18:54:37.693825: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def pre_emphasis(signal,factor=0.97):
    return np.append(signal[0], signal[1:] - factor * signal[:-1])

In [3]:
def inverse_filtering(signal, lpc_coeffs):
    return lfilter(np.concatenate(([1], -lpc_coeffs[1:])), [1], signal)

In [4]:
def compute_lpc(signal, order):
    autocorr = correlate(signal, signal, mode='full')
    autocorr = autocorr[len(signal)-1:len(signal)+order]
    
    a = np.concatenate(([1], -autocorr[1:order+1]))
    b = [1]
    
    lpc_coeffs = lfilter(b, a, signal)
    
    lpc_coeffs = np.concatenate(([1], -lpc_coeffs[1:order+1]))
    
    del autocorr,a,b
    
    return lpc_coeffs

In [5]:
def lpc_to_cepstrum(lpc_coeffs):
    return np.fft.ifft(np.log(np.abs(np.fft.fft(lpc_coeffs))))

In [6]:
import gc
from IPython.display import clear_output

def create_spectrograms(train_folder, create_folder, verbose=False, speakers=50, utterances=10, sr=7000, frame_length=2048, hop_length=512, lpc_order=16):
    spc_folder = os.path.join(create_folder, "spectrogram")
    vt_folder = os.path.join(create_folder, "vocal_tract")
    glot_folder = os.path.join(create_folder, "glottal")
    
    total_speaker = 0
    for speaker in os.listdir(train_folder):
        total_utterances = 0
        speaker_folder_spc = os.path.join(spc_folder, speaker)
        speaker_folder_vt = os.path.join(vt_folder, speaker)
        speaker_folder_glot = os.path.join(glot_folder, speaker)
        os.makedirs(speaker_folder_spc, exist_ok=True)
        os.makedirs(speaker_folder_vt, exist_ok=True)
        os.makedirs(speaker_folder_glot, exist_ok=True)
        
        for vidID in os.listdir(os.path.join(train_folder, speaker)):
            for file in os.listdir(os.path.join(train_folder, speaker, vidID)):
                if file.endswith(".wav"):
                    wav_file_path = os.path.join(train_folder, speaker, vidID, file)
                    
                    y, sr = librosa.load(wav_file_path, sr=sr)
                    
#                     y_preemphasized=pre_emphasis(y)
                    y_preemphasized=y
                    
                    frames = librosa.util.frame(y_preemphasized, frame_length=frame_length, hop_length=hop_length).T
                    del y_preemphasized
                    
                    window = np.hamming(frame_length)
                    frames_windowed = frames * window  
                    del frames
                    
                    lpc_coeffs = []
                    for frame in frames_windowed:
                        frame=np.array(frame)
                        coeff = compute_lpc(frame, lpc_order)
                        lpc_coeffs.append(coeff)
                        
                    glottal_waveforms = [inverse_filtering(frame, coeff) for frame, coeff in zip(frames_windowed, lpc_coeffs)]
                    glottal_waveforms_avg = np.mean(np.abs(np.array(glottal_waveforms)), axis=0)
                    del glottal_waveforms
                    
                    residuals = []
                    for i, coeff in enumerate(lpc_coeffs):
                        frame = frames_windowed[i]
                        residual = lfilter(coeff, [1.0], frame)
                        residuals.append(residual)
                        
                    residual_avg = np.mean(np.abs(np.array(residuals)), axis=0)
                    
                    del frames_windowed, lpc_coeffs, residuals
                    gc.collect()

                    try:
                        plt.figure(figsize=(5, 5))
                        ax = plt.axes()
                        ax.set_axis_off()
                        plt.set_cmap('hot')
                        amp=np.abs(librosa.stft(y))
                        D = librosa.amplitude_to_db(amp, ref=np.max)
                        del y,amp
                        librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
                        output_path = os.path.join(speaker_folder_spc, "spc"+str(total_utterances+1) + ".png")
                        plt.savefig(output_path, bbox_inches='tight', transparent=True, pad_inches=0.0)
                        plt.close('all')
                        del D,ax,output_path
                        gc.collect()

                        plt.figure(figsize=(5, 5))
                        ax = plt.axes()
                        ax.set_axis_off()
                        plt.set_cmap('hot')
                        amp=np.abs(librosa.stft(residual_avg))
                        D_residual = librosa.amplitude_to_db(amp, ref=np.max)
                        del amp
                        librosa.display.specshow(D_residual, sr=sr, x_axis='time', y_axis='log')
                        output_path = os.path.join(speaker_folder_vt, "vt"+str(total_utterances+1) + ".png")
                        plt.savefig(output_path, bbox_inches='tight', transparent=True, pad_inches=0.0)
                        plt.close('all')
                        del D_residual,ax,output_path
                        gc.collect()

                        plt.figure(figsize=(5, 5))
                        ax = plt.axes()
                        ax.set_axis_off()
                        plt.set_cmap('hot')
                        amp=np.abs(librosa.stft(glottal_waveforms_avg))
                        D_glot = librosa.amplitude_to_db(amp, ref=np.max)
                        del amp
                        librosa.display.specshow(D_glot, sr=sr, x_axis='time', y_axis='log')
                        output_path = os.path.join(speaker_folder_glot, "glot"+str(total_utterances+1) + ".png")
                        plt.savefig(output_path, bbox_inches='tight', transparent=True, pad_inches=0.0)
                        plt.close('all')
                        del D_glot,ax,output_path
                        gc.collect()
                        
#                         clear_output()
                    
                    except OSError as e:
                        print(f"Error saving spectrogram for {wav_file_path}: {e}")
                    
                    del residual_avg, glottal_waveforms_avg
                    gc.collect()
    
                    total_utterances += 1
                    if total_utterances == utterances:
                        break
            
            if total_utterances == utterances:
                break
        
        total_speaker += 1
        if total_speaker != 0 and total_speaker % 1 == 0 and verbose:
            print(f"{total_speaker} speakers completed.\n")
        
        if total_speaker == speakers:
            break

In [7]:
raw_dataset_loc="/kaggle/input/voxceleb1train/wav"
save_loc="/kaggle/working/"
num_speakers=10
num_utterances=100

In [8]:
create_spectrograms(raw_dataset_loc,save_loc,True,num_speakers,num_utterances)

1 speakers completed.

2 speakers completed.

3 speakers completed.

4 speakers completed.

5 speakers completed.

6 speakers completed.

7 speakers completed.

8 speakers completed.

9 speakers completed.

10 speakers completed.



In [9]:
dataset_path_spc="/kaggle/working/spectrogram"
dataset_path_vt="/kaggle/working/vocal_tract"
dataset_path_glot="/kaggle/working/glottal"
folders=os.listdir(dataset_path_spc)

In [10]:
dict={}
i=0
for val in folders:
    dict[val]=i
    i=i+1

dict

{'id10719': 0,
 'id10484': 1,
 'id10480': 2,
 'id10036': 3,
 'id11123': 4,
 'id10061': 5,
 'id10459': 6,
 'id11079': 7,
 'id10116': 8,
 'id11250': 9}

In [11]:
dataset_path_spc="/kaggle/working/spectrogram"
dataset_path_vt="/kaggle/working/vocal_tract"
dataset_path_glot="/kaggle/working/glottal"
folders=os.listdir(dataset_path_spc)

# Initializing training and test dataset
X_train=[[],[],[]]
y_train=[[],[],[]]
X_test=[[],[],[]]
y_test=[[],[],[]]

# Split the dataset into training and test set.
num=np.random.rand(num_utterances*num_speakers)
mask=num<0.2
split=mask.astype(int)

for dirs in folders:
    i=0
    for img in os.listdir(os.path.join(dataset_path_spc,dirs)):
        image=Image.open(os.path.join(dataset_path_spc,dirs,img))
        new_img=image.resize((200,200))
        tmp_array=np.array(new_img)/255.
        if split[i]==0:
            X_train[0].append(tmp_array)
            y_train[0].append(dict[str(dirs)])
        else:
            X_test[0].append(tmp_array)
            y_test[0].append(dict[str(dirs)])
            
        i+=1
        
    i=0    
    for img in os.listdir(os.path.join(dataset_path_vt,dirs)):
        image=Image.open(os.path.join(dataset_path_vt,dirs,img))
        new_img=image.resize((200,200))
        tmp_array=np.array(new_img)/255.
        if split[i]==0:
            X_train[1].append(tmp_array)
            y_train[1].append(dict[str(dirs)])
        else:
            X_test[1].append(tmp_array)
            y_test[1].append(dict[str(dirs)])
            
        i+=1
            
    i=0
    for img in os.listdir(os.path.join(dataset_path_glot,dirs)):
        image=Image.open(os.path.join(dataset_path_glot,dirs,img))
        new_img=image.resize((200,200))
        tmp_array=np.array(new_img)/255.
        if split[i]==0:
            X_train[2].append(tmp_array)
            y_train[2].append(dict[str(dirs)])
        else:
            X_test[2].append(tmp_array)
            y_test[2].append(dict[str(dirs)])
        
        i+=1

In [12]:
def create_train_triplets(X_train, y_train, num_triplets):
    triplets = []
    labels = np.unique(y_train)
    
    for _ in range(num_triplets):
        anchor_label = np.random.choice(labels)
        negative_label = np.random.choice(labels[labels != anchor_label])
        
        anchor_indices = np.where(y_train == anchor_label)[0]
        positive_indices = np.where(y_train == anchor_label)[0]
        negative_indices = np.where(y_train == negative_label)[0]
        
        anchor = X_train[np.random.choice(anchor_indices)]
        positive = X_train[np.random.choice(positive_indices)]
        negative = X_train[np.random.choice(negative_indices)]
        
        triplets.append((anchor, positive, negative))
    
    return np.array(triplets)

In [13]:
def create_test_triplets(X_test, y_test, num_triplets):
    triplets = []
    range_test = len(y_test)
    idx = np.arange(range_test)
    
    for _ in range(num_triplets):
        label_1 = np.random.choice(idx)
        label_2 = np.random.choice(idx)
        
        anchor = X_test[label_1]
        inp = X_test[label_2]
        out = 1 if y_test[label_1] == y_test[label_2] else 0
        
        triplets.append((anchor, inp, out))
        
    return np.array(triplets, dtype=object)  

In [14]:
def convolutional_model(input_shape):
    input_img = tf.keras.Input(shape=input_shape)
    Z1=tfl.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same')(input_img)
    A1=tfl.ReLU()(Z1)
    P1=tfl.MaxPool2D(pool_size=(4,4),padding='same')(A1)
    Z2=tfl.Conv2D(filters=64,kernel_size=(2,2),strides=(1,1),padding='same')(P1)
    A2=tfl.ReLU()(Z2)
    P2=tfl.MaxPool2D(pool_size=(4,4),padding='same')(A2)
    Z3=tfl.Conv2D(filters=128,kernel_size=(2,2),strides=(1,1),padding='same')(P2)
    A3=tfl.ReLU()(Z3)
    P3=tfl.MaxPool2D(pool_size=(4,4),padding='same')(A3)
    F=tfl.Flatten()(P3)
    D1=tfl.Dense(128)(F)
        
    model = tf.keras.Model(inputs=input_img, outputs=D1)
    return model

input_shape = (200, 200, 4)
base_model = convolutional_model(input_shape)

In [42]:
def triplet_loss(y_true, y_pred, margin=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
    return tf.reduce_mean(loss)

In [43]:
anchor_input = tfl.Input(shape=input_shape, name='anchor_input')
positive_input = tfl.Input(shape=input_shape, name='positive_input')
negative_input = tfl.Input(shape=input_shape, name='negative_input')

encoded_anchor = base_model(anchor_input)
encoded_positive = base_model(positive_input)
encoded_negative = base_model(negative_input)

triplet_model_spc = tf.keras.Model(inputs=[anchor_input, positive_input, negative_input],
                      outputs=[encoded_anchor, encoded_positive, encoded_negative])

triplet_model_spc.compile(optimizer='adam', loss=triplet_loss)

triplet_model_vt = tf.keras.Model(inputs=[anchor_input, positive_input, negative_input],
                      outputs=[encoded_anchor, encoded_positive, encoded_negative])

triplet_model_vt.compile(optimizer='adam', loss=triplet_loss)

triplet_model_glot = tf.keras.Model(inputs=[anchor_input, positive_input, negative_input],
                      outputs=[encoded_anchor, encoded_positive, encoded_negative])

triplet_model_glot.compile(optimizer='adam', loss=triplet_loss)

In [44]:
train_num=500
test_num=100

In [55]:
triplets_train=create_train_triplets(X_train[0],y_train[0],train_num)

anchor_images = triplets_train[:, 0]
positive_images = triplets_train[:, 1]
negative_images = triplets_train[:, 2]

triplet_model_spc.fit([anchor_images, positive_images, negative_images], 
                  np.zeros((train_num, )),
                  batch_size=32, 
                  epochs=100)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.2014
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.1991
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2014
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1984
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2022
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1974
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2018
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1937
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2352
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2036
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2098
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2006
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1999
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2013
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - lo

In [62]:
triplets_test=create_test_triplets(X_test[0],y_test[0],test_num)

# anchor_images = triplets_test[:, 0]
# input_images = triplets_test[:, 1]
# label_test_spc = triplets_test[:, 2]

true_pos = 0
false_pos = 0

anchor_input_test = np.stack(triplets_test[:, 0])
positive_input_test = np.stack(triplets_test[:, 1])
labels_test = triplets_test[:, 2]

# print(np.sum(labels_test))

encoded_anchor = base_model.predict(anchor_input_test)
encoded_positive = base_model.predict(positive_input_test)

distances = np.linalg.norm(encoded_anchor - encoded_positive, axis=1)
threshold = 0.035

for i in range(test_num):
    print(str(distances[i])+" : "+str(labels_test[i]))
    if distances[i] < threshold and labels_test[i] == 1:
        true_pos += 1
    if distances[i] >= threshold and labels_test[i] == 0:
        false_pos += 1

total_pos=np.sum(labels_test)
true_pos = (true_pos) / total_pos
accuracy = (false_pos) / (100-total_pos)
accuracy = (true_pos+false_pos) / test_num
print("True Positives: {:.2f}".format(true_pos))
print("False Positives: {:.2f}".format(false_pos))
print("Accuracy: {:.2f}".format(accuracy))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
0.032571193 : 0
0.037016958 : 0
0.043218892 : 0
0.023613397 : 0
0.059122503 : 0
0.032289002 : 0
0.04088972 : 1
0.03765131 : 0
0.045791287 : 0
0.028783264 : 0
0.035669256 : 0
0.04105132 : 0
0.022111414 : 1
0.0582372 : 0
0.031108191 : 1
0.027589174 : 0
0.044402637 : 0
0.029262763 : 0
0.022181422 : 0
0.028568786 : 0
0.029759008 : 0
0.030490225 : 0
0.037451606 : 0
0.03692642 : 0
0.042611986 : 0
0.041650094 : 0
0.023745017 : 0
0.045721907 : 0
0.045386862 : 0
0.033265322 : 0
0.047773097 : 0
0.022235457 : 1
0.038447358 : 0
0.053737298 : 0
0.04920475 : 0
0.025933221 : 0
0.028949047 : 0
0.023438344 : 1
0.047401737 : 1
0.062881134 : 0
0.037923608 : 0
0.038056847 : 0
0.043484617 : 0
0.034837235 : 1
0.06520811 : 0
0.033105943 : 0
0.036763355 : 0
0.03189815 : 0
0.029820666 : 0
0.026924431 : 1
0.037927188 : 0
0.019818576 : 0
0.046826843 : 1
0.03926319 : 1
0.060523182 : 0
0.053237792 : 0
0.0 : 1
0.029415615 : 0
0.030430779 : 

In [63]:
triplets_train=create_train_triplets(X_train[1],y_train[1],train_num)

anchor_images = triplets_train[:, 0]
positive_images = triplets_train[:, 1]
negative_images = triplets_train[:, 2]

triplet_model_vt.fit([anchor_images, positive_images, negative_images], 
                  np.zeros((train_num, )),
                  batch_size=32, 
                  epochs=100)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.1995
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1999
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1990
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1985
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2061
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2041
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2000
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2004
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2010
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1999
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2001
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1996
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2000
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2002
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - lo

In [66]:
triplets_test=create_test_triplets(X_test[1],y_test[1],test_num)

# anchor_images = triplets_test[:, 0]
# input_images = triplets_test[:, 1]
# label_test_spc = triplets_test[:, 2]

true_pos = 0
false_pos = 0

anchor_input_test = np.stack(triplets_test[:, 0])
positive_input_test = np.stack(triplets_test[:, 1])
labels_test = triplets_test[:, 2]

# print(np.sum(labels_test))

encoded_anchor = base_model.predict(anchor_input_test)
encoded_positive = base_model.predict(positive_input_test)

distances = np.linalg.norm(encoded_anchor - encoded_positive, axis=1)
threshold = 0.033

for i in range(test_num):
    print(str(distances[i])+" : "+str(labels_test[i]))
    if distances[i] < threshold and labels_test[i] == 1:
        true_pos += 1
    if distances[i] >= threshold and labels_test[i] == 0:
        false_pos += 1

total_pos=np.sum(labels_test)
true_pos = (true_pos) / total_pos
accuracy = (false_pos) / (100-total_pos)
accuracy = (true_pos+false_pos) / test_num
print("True Positives: {:.2f}".format(true_pos))
print("False Positives: {:.2f}".format(false_pos))
print("Accuracy: {:.2f}".format(accuracy))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
0.041942306 : 0
0.029673168 : 0
0.047374796 : 0
0.036252394 : 0
0.024885274 : 0
0.050208285 : 0
0.051663812 : 0
0.022876281 : 0
0.06391047 : 0
0.02091957 : 0
0.0534198 : 0
0.049958885 : 0
0.036229827 : 0
0.0 : 1
0.048393674 : 0
0.027946007 : 0
0.030052833 : 0
0.03409557 : 0
0.04810068 : 0
0.032881804 : 0
0.05602186 : 0
0.02821392 : 1
0.031528763 : 0
0.063395984 : 1
0.049978953 : 0
0.024426552 : 0
0.047585923 : 0
0.040297046 : 0
0.03551462 : 0
0.04927551 : 0
0.044993956 : 0
0.04070964 : 1
0.027675433 : 1
0.03056455 : 0
0.039857097 : 0
0.028453086 : 0
0.034590192 : 0
0.026163246 : 0
0.032282595 : 1
0.02651704 : 1
0.038933843 : 0
0.0245754 : 0
0.024938513 : 0
0.027764821 : 0
0.032063268 : 0
0.044794206 : 0
0.038213335 : 0
0.028850736 : 1
0.032758273 : 0
0.0169083 : 0
0.03623373 : 0
0.015130632 : 0
0.048882313 : 0
0.031597786 : 0
0.03223983 : 1
0.023988992 : 0
0.02425792 : 0
0.06096614 : 0
0.03573106 : 0
0.02878705

In [67]:
triplets_train=create_train_triplets(X_train[2],y_train[2],train_num)

anchor_images = triplets_train[:, 0]
positive_images = triplets_train[:, 1]
negative_images = triplets_train[:, 2]

triplet_model_glot.fit([anchor_images, positive_images, negative_images], 
                  np.zeros((train_num, )),
                  batch_size=32, 
                  epochs=100)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.2006
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.2003
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2000
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2008
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2000
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1997
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1994
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.2008
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2068
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1981
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1994
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2040
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2001
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1991
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - lo

In [72]:
triplets_test=create_test_triplets(X_test[2],y_test[2],test_num)

# anchor_images = triplets_test[:, 0]
# input_images = triplets_test[:, 1]
# label_test_spc = triplets_test[:, 2]

true_pos = 0
false_pos = 0

anchor_input_test = np.stack(triplets_test[:, 0])
positive_input_test = np.stack(triplets_test[:, 1])
labels_test = triplets_test[:, 2]

# print(np.sum(labels_test))

encoded_anchor = base_model.predict(anchor_input_test)
encoded_positive = base_model.predict(positive_input_test)

distances = np.linalg.norm(encoded_anchor - encoded_positive, axis=1)
threshold = 0.04

for i in range(test_num):
    print(str(distances[i])+" : "+str(labels_test[i]))
    if distances[i] < threshold and labels_test[i] == 1:
        true_pos += 1
    if distances[i] >= threshold and labels_test[i] == 0:
        false_pos += 1

total_pos=np.sum(labels_test)
true_pos = (true_pos) / total_pos
accuracy = (false_pos) / (100-total_pos)
accuracy = (true_pos+false_pos) / test_num
print("True Positives: {:.2f}".format(true_pos))
print("False Positives: {:.2f}".format(false_pos))
print("Accuracy: {:.2f}".format(accuracy))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
0.036462482 : 0
0.046690173 : 0
0.03631498 : 0
0.046300955 : 0
0.052497387 : 0
0.028485507 : 0
0.045323417 : 0
0.0 : 1
0.046280675 : 0
0.03797783 : 0
0.07449623 : 0
0.03371066 : 0
0.077440105 : 0
0.076899596 : 0
0.05482457 : 0
0.037286926 : 0
0.026123162 : 0
0.034338452 : 0
0.037981153 : 0
0.04677708 : 0
0.044319853 : 0
0.02556194 : 0
0.041003164 : 0
0.03864629 : 0
0.0303962 : 0
0.028046863 : 0
0.07525096 : 0
0.03956226 : 0
0.050166305 : 1
0.060230203 : 0
0.041774437 : 0
0.07143728 : 0
0.06212443 : 0
0.036124974 : 0
0.024490088 : 0
0.031843748 : 0
0.06613301 : 0
0.037441935 : 0
0.04620837 : 0
0.08188273 : 0
0.034286737 : 0
0.025472283 : 0
0.06383243 : 0
0.053273235 : 0
0.03898377 : 0
0.047468238 : 0
0.040030275 : 0
0.058106728 : 0
0.029785313 : 0
0.07715926 : 0
0.06498849 : 0
0.05227972 : 0
0.04374452 : 0
0.0 : 1
0.05131239 : 0
0.038815055 : 0
0.048435904 : 0
0.024581464 : 0
0.02790749 : 0
0.027940782 : 0
0.027

In [ ]:
# import zipfile

# source_path = '/kaggle/working'  
# zip_filename = 'saved_data.zip' 

# with zipfile.ZipFile(zip_filename, 'w') as zipf:

#     for root, _, files in os.walk(source_path):
#         for file in files:
#             zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), source_path))
